In [10]:
import math
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from model import aagcn, loss, SAM
from utils import adj_mat, training_supervision
from data.handpose_dataset import HandPoseDatasetNumpy, df_to_numpy
from data.get_data_from_csv import get_train_data, get_val_data
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import sklearn
from config_fine_tuning_occluded_hand_detection import CFG

Import model

In [2]:
model_path = 'trained_models/7_AAGCN_Focal_seqlen32_release_SAM_joints1_joints2_oridist/f10.8142688679245284_valloss310.2437744140625_epoch13.pth'

graph = aagcn.Graph(adj_mat.num_node, adj_mat.self_link, adj_mat.inward, adj_mat.outward, adj_mat.neighbor)

model = aagcn.Model(num_class=CFG.num_classes, num_point=21, num_person=1, graph=graph, drop_out=0.5,
                    in_channels=CFG.num_feats)

checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cuda


Model(
  (data_bn): BatchNorm1d(63, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l1): TCN_GCN_unit(
    (gcn1): unit_gcn(
      (conv_d): ModuleList(
        (0-2): 3 x Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (conv_a): ModuleList(
        (0-2): 3 x Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (conv_b): ModuleList(
        (0-2): 3 x Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (conv_ta): Conv1d(64, 1, kernel_size=(9,), stride=(1,), padding=(4,))
      (conv_sa): Conv1d(64, 1, kernel_size=(21,), stride=(1,), padding=(10,))
      (fc1c): Linear(in_features=64, out_features=32, bias=True)
      (fc2c): Linear(in_features=32, out_features=64, bias=True)
      (down): Sequential(
        (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

## Modify model -> 3 classes

In [3]:
num_classes_fine_tuning = len(CFG.classes_fine_tuning)

# change last layer
model.fc = nn.Linear(128, num_classes_fine_tuning).to(device)

# init the new layer
nn.init.normal_(model.fc.weight, 0, math.sqrt(2. / num_classes_fine_tuning))

Parameter containing:
tensor([[-0.3934, -1.2816, -0.2832,  0.0083, -1.4112, -0.6198,  0.2116, -0.8508,
          0.0153,  1.6623,  0.2825,  0.4693,  0.9603,  0.1855, -1.1737, -0.3825,
         -0.8257,  1.4569,  1.7515,  0.2982, -1.1137, -1.3337,  0.3068, -0.4123,
         -0.4261,  0.9741,  0.7653, -0.2347,  0.3605,  0.1354,  0.0198, -1.8644,
          0.2086, -1.2205, -0.6561,  1.7379, -0.7430,  0.5457,  0.3010, -0.4721,
         -1.7002,  0.9229, -0.9523,  0.7504,  0.6512,  0.4300,  1.7054,  0.5913,
         -0.1570, -0.1880, -0.2956, -0.1273, -0.7981, -1.1487, -1.0729, -0.0380,
          0.8234,  0.7685,  0.8407,  0.7783, -0.3994, -0.1769, -0.0188,  1.1419,
         -1.1534,  0.2771,  0.0415, -0.5472, -0.1558, -0.2133, -1.9954,  0.8369,
         -0.9280,  0.0053, -0.1966, -0.2587,  0.7253,  0.3017, -0.4302,  0.1958,
          0.6542, -0.3581,  1.3611,  0.6461,  0.1558, -1.3570,  0.6970,  0.2455,
         -1.1545,  0.5867, -0.1225, -0.8771, -0.2988,  0.3984,  0.0684, -0.4869,
      

Get data

In [4]:
df_train = get_train_data()  # Todo: replace with our own data
df_train = df_train.replace("Postion", "Position")
df_val = get_val_data()  # Todo: replace with our own data

df_train.head()

,WRIST,THUMB_CMC,THUMB_MCP,THUMB_IP,THUMB_TIP,INDEX_FINGER_MCP,INDEX_FINGER_PIP,INDEX_FINGER_DIP,INDEX_FINGER_TIP,MIDDLE_FINGER_MCP,...,MIDDLE_FINGER_TIP,RING_FINGER_MCP,RING_FINGER_PIP,RING_FINGER_DIP,RING_FINGER_TIP,PINKY_MCP,PINKY_PIP,PINKY_DIP,PINKY_TIP,LABEL
0,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",Negative
1,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",Negative
2,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",Negative
3,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",Negative
4,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",Negative


Finetuning first try: remove all other classes except our 3 from the data

In [5]:
df_train = df_train[df_train["LABEL"].isin(CFG.classes_fine_tuning)]
df_val = df_val[df_val["LABEL"].isin(CFG.classes_fine_tuning)]
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27184 entries, 0 to 2021
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   WRIST              27184 non-null  object
 1   THUMB_CMC          27184 non-null  object
 2   THUMB_MCP          27184 non-null  object
 3   THUMB_IP           27184 non-null  object
 4   THUMB_TIP          27184 non-null  object
 5   INDEX_FINGER_MCP   27184 non-null  object
 6   INDEX_FINGER_PIP   27184 non-null  object
 7   INDEX_FINGER_DIP   27184 non-null  object
 8   INDEX_FINGER_TIP   27184 non-null  object
 9   MIDDLE_FINGER_MCP  27184 non-null  object
 10  MIDDLE_FINGER_PIP  27184 non-null  object
 11  MIDDLE_FINGER_DIP  27184 non-null  object
 12  MIDDLE_FINGER_TIP  27184 non-null  object
 13  RING_FINGER_MCP    27184 non-null  object
 14  RING_FINGER_PIP    27184 non-null  object
 15  RING_FINGER_DIP    27184 non-null  object
 16  RING_FINGER_TIP    27184 non-null  object
 17 

Prepare train and validation data

In [6]:
print("[INFO] TRAIN DATA DISTRIBUTION")
print(df_train["LABEL"].value_counts())
print("[INFO] VALIDATION DATA DISTRIBUTION")
print(df_val["LABEL"].value_counts())

train_numpy = df_to_numpy(df_train)
val_numpy = df_to_numpy(df_val)

train_set = HandPoseDatasetNumpy(train_numpy)
val_set = HandPoseDatasetNumpy(val_numpy)

train_loader = DataLoader(train_set, batch_size=CFG.batch_size, drop_last=True, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=CFG.batch_size, drop_last=True, pin_memory=True)

print(f"[INFO] TRAINING ON {len(train_set)} DATAPOINTS")
print(f"[INFO] VALIDATION ON {len(val_set)} DATAPOINTS")

[INFO] TRAIN DATA DISTRIBUTION
LABEL
Grasp       19445
Negative     5011
Release      2728
Name: count, dtype: int64
[INFO] VALIDATION DATA DISTRIBUTION
LABEL
Grasp       3679
Negative    1117
Release      380
Name: count, dtype: int64
[INFO] TRAINING ON 27184 DATAPOINTS
[INFO] VALIDATION ON 5176 DATAPOINTS


Fine tuning config and preparation

In [7]:
optimizer_base = torch.optim.Adam
optimizer = SAM.SAM(model.parameters(), optimizer_base, lr=CFG.lr, rho=0.5, adaptive=True)
criterion = loss.FocalLoss()

writer = SummaryWriter(f'fine_tuned_models_occluded_hand_detection/runs/{CFG.experiment_name}')

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=CFG.min_lr)

In [ ]:
def train_func(model, data_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    iters = len(data_loader)
    global_step = epoch * len(data_loader)
    preds = []
    groundtruth = []
    t0 = time.time()
    loss_total = 0
    for i, (inputs, labels) in enumerate(data_loader):
        labels = labels.cuda().long()
        inputs = inputs.cuda().float()

        last_label = labels[:, -1, :]
        last_label = torch.argmax(last_label, 1)

        model.zero_grad()
        last_out = model(inputs)

        # first forward-backward pass
        loss = criterion(last_out, last_label)
        loss.backward()

        if CFG.sam:
            optimizer.first_step(zero_grad=True)  #

            # second forward-backward pass
            criterion(model(inputs), last_label).backward()  #
            optimizer.second_step(zero_grad=True)  #
        else:
            optimizer.step()

        current_lr = optimizer.param_groups[0]['lr']

        preds.append(last_out.cpu().detach().numpy())
        groundtruth.append(last_label.cpu().detach().numpy())

        loss_total += loss
        global_step += 1
        writer.add_scalar('Loss/Train', loss, global_step)
        writer.add_scalar('LR', current_lr, global_step)

        if i % CFG.print_freq == 1 or i == iters - 1:
            t1 = time.time()
            print(
                f"[TRAIN] Epoch: {epoch}/{CFG.epochs} | Iteration: {i}/{iters} | Loss: {loss_total / i} | LR: {current_lr} | ETA: {((t1 - t0) / i * iters) - (t1 - t0)}s")

    return loss_total, np.argmax(preds, axis=2).flatten(), np.array(groundtruth).flatten()


def eval_func(model, criterion, data_loader, epoch):
    model.eval()
    preds = []
    groundtruth = []
    t0 = time.time()
    loss_total = 0
    global_step = len(train_loader) * epoch
    iters = len(data_loader)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(data_loader):
            labels = labels.cuda().long()
            inputs = inputs.cuda().float()

            last_label = labels[:, -1, :]
            last_label = torch.argmax(last_label, 1)

            last_out = model(inputs)
            loss = criterion(last_out, last_label)

            preds.append(last_out.cpu().detach().numpy())
            groundtruth.append(last_label.cpu().detach().numpy())
            loss_total += loss

            if i % CFG.print_freq == 1 or i == iters - 1:
                t1 = time.time()
                print(
                    f"[EVAL] Epoch: {epoch}/{CFG.epochs} | Iteration: {i}/{iters} | Val-Loss: {loss_total / i} | ETA: {((t1 - t0) / i * iters) - (t1 - t0)}s")

    writer.add_scalar('Loss/Validation', loss_total / i, global_step)
    return loss_total, np.argmax(preds, axis=2).flatten(), np.array(groundtruth).flatten()

Early stopping to stop training automatically when model doesn't improve anymore

In [8]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


early_stopping = EarlyStopping(patience=10, delta=0.001)

Start Fine tuning

In [9]:
start_epoch = 0

for epoch in range(start_epoch, CFG.epochs + start_epoch):
    global_step = len(train_loader) * epoch

    #TRAIN
    train_loss, preds_train, gt_train = train_func(model, train_loader, criterion, optimizer, scheduler, epoch)
    train_grad_flow_plot = training_supervision.get_plot_grad_flow(model)

    f1_train = f1_score(gt_train, preds_train, average="micro")
    writer.add_scalar('Accuracy/Train', f1_train, global_step)
    print(f"[TRAIN] Training F1-Score {f1_train}")

    #Model Gradients
    names, gradmean = training_supervision.get_model_grads(model)
    _limits = np.array([float(i) for i in range(len(gradmean))])
    _num = len(gradmean)
    writer.add_histogram_raw(tag="ModelGrads/MeanGradientFlow", min=0.0, max=0.5, num=_num,
                             sum=gradmean.sum(), sum_squares=np.power(gradmean, 2).sum(), bucket_limits=_limits,
                             bucket_counts=gradmean, global_step=global_step)

    #VAL
    val_loss, preds_val, gt_val = eval_func(model, criterion, val_loader, epoch)

    f1_val_micro = f1_score(gt_val, preds_val, average="micro")
    f1_val_macro = f1_score(gt_val, preds_val, average="macro")
    writer.add_scalar('Accuracy/Validation/F1-Micro', f1_val_micro, global_step)
    writer.add_scalar('Accuracy/Validation/F1-Macro', f1_val_macro, global_step)
    print(f"[EVAL] Validation F1-Score Micro {f1_val_micro}")
    print(f"[EVAL] Validation F1-Score Macro {f1_val_macro}")

    #Conf Mat
    cm = sklearn.metrics.confusion_matrix(gt_val, preds_val)
    cm_plot = training_supervision.plot_confusion_matrix(cm, CFG.classes)
    writer.add_figure("Confusion Matrix/Validation", cm_plot, global_step)

    #Model Weights
    names, params = training_supervision.get_model_weights(model)
    for n, p in zip(names, params):
        writer.add_histogram(f"ModelWeights/{n}", p, global_step)

    print("[EVAL] Classification Report")
    print(classification_report(gt_val, preds_val, target_names=CFG.classes, digits=3))

    scheduler.step(val_loss)  #for reduce lr on plateau

    PATH = f"fine_tuned_models_occluded_hand_detection/{CFG.experiment_name}/f1{f1_val_micro}_valloss{val_loss}_epoch{epoch}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'f1_micro_val-score': f1_val_micro,
    }, PATH)
    print("[INFO] MODEL SAVED")
    
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
